# Import

In [12]:
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, normalize
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Classe per la classificazione dei supravvisuti al Titanic
La classe utilizza la classe `sklearn.neural_network.MLPClassifier` per la classificazione dei passeggeri del Titanic.
Il modello è stato addestrato utilizzando il dataset `titanic.csv` scaricato dat [Kaagle](https://www.kaggle.com/datasets/brendan45774/test-file).
Dato che il dataset non ha tutte le colonne piene sono state effettuati delle procedure di riempiemento dei valori mancanti utilizzando al classe `sklearn.impute.SimpleImputer` per riempire i valori mancanti con i valori più frequenti.
Inoltre utilizzando la classe `sklearn.preprocessing.LabelEncoder` sono state trasformate le colonne categoriche in colonne numeriche.

In [13]:
class TitanicSurvivalClassifier:
    def __init__(self, data_path):
        self.data = pd.read_csv(data_path)
        self.label_encoder = LabelEncoder()
        # Inizializiamo l'Imputer in modo che inserisca i valori più fequenti delle celle delle colonne dove mancano i dati
        self.imputer = SimpleImputer(strategy='most_frequent')
        self.model = None
        self.X = None
        self.y = None

    def preprocess_data(self):
        # Rimuoviamo la colonna Cabin dato che è per il 78% null
        self.data.drop(columns=['Cabin'], inplace=True)

        # Riempiamo i valori mancanti nella colonna 'Age', e 'Fare'
        # Stiamo utilizzando un imputer di tipo SimpleImputer, questa classe 
        self.data['Age'] = self.imputer.fit_transform(self.data[['Age']])
        self.data['Fare'] = self.imputer.fit_transform(self.data[['Fare']])

        # Codifica le variabili categoriche
        self.data['Sex'] = self.label_encoder.fit_transform(self.data['Sex'])
        self.data['Embarked'] = self.label_encoder.fit_transform(self.data['Embarked'])

        # Seleziona le feature e il target
        self.X = self.data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
        self.y = self.data['Survived']
        print(self.X)

    def train_model(self):
        # Dividi il dataset in training e test set
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)
        
        # Inizializza il modello MLPClassifier
        self.model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)

        # Addestra il modello
        self.model.fit(X_train, y_train)

        # Valuta l'accuratezza del modello
        train_accuracy = accuracy_score(y_train, self.model.predict(X_train))
        test_accuracy = accuracy_score(y_test, self.model.predict(X_test))
        print(f"Training Accuracy: {train_accuracy}")
        print(f"Test Accuracy: {test_accuracy}")

    def predict(self, passenger_data):
        # Effettua una previsione per nuovi dati di passeggeri
        return self.model.predict(passenger_data)

In [14]:
# Percorso del file CSV
data_path = "./datasets/titanic.csv"

# Crea un'istanza del classificatore TitanicSurvivalClassifier
classifier = TitanicSurvivalClassifier(data_path)

# Preelabora i dati
classifier.preprocess_data()

# Addestra il modello
classifier.train_model()

new_passenger_data = pd.DataFrame([[3, 1, 25, 0, 0, 7.5, 2]], columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])
new_passenger_data


     Pclass  Sex   Age  SibSp  Parch      Fare  Embarked
0         3    1  34.5      0      0    7.8292         1
1         3    0  47.0      1      0    7.0000         2
2         2    1  62.0      0      0    9.6875         1
3         3    1  27.0      0      0    8.6625         2
4         3    0  22.0      1      1   12.2875         2
..      ...  ...   ...    ...    ...       ...       ...
413       3    1  21.0      0      0    8.0500         2
414       1    0  39.0      0      0  108.9000         0
415       3    1  38.5      0      0    7.2500         2
416       3    1  21.0      0      0    8.0500         2
417       3    1  21.0      1      1   22.3583         0

[418 rows x 7 columns]
Training Accuracy: 1.0
Test Accuracy: 1.0


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,25,0,0,7.5,2


In [15]:
prediction = classifier.predict(new_passenger_data)
print("Survived" if prediction == 1 else "Dead")

Dead
